In [1]:
import tensorflow as tf 
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow.keras as keras
import tensorflow.keras.layers as layers 
import time

In [2]:
discriminator2 = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 2)),
        layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.GlobalMaxPooling2D(),
        layers.Dense(1),
    ],
    name="discriminator",
)
input1=keras.Input([28,28,1])
input2=keras.Input([10,])
net=keras.layers.Dense(128)(input2)
net=keras.layers.LeakyReLU(0.2)(net)
net=keras.layers.Dense(28*28)(net)
net=keras.layers.Reshape((28,28,1))(net)
net=keras.layers.concatenate([input1,net])
out=discriminator2(net)
discriminator=keras.Model(inputs=[input1,input2],outputs=out)
latent_dim = 32

generator = keras.Sequential(
    [
        keras.Input(shape=(latent_dim+10,)),
        # We want to generate 128 coefficients to reshape into a 7x7x128 map
        layers.Dense(7 * 7 * 128),
        layers.LeakyReLU(alpha=0.2),
        layers.Reshape((7, 7, 128)),
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128,4,1,padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128,4,1,padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(1, (7, 7), padding="same", activation="sigmoid"),
    ],
    name="generator",
)

In [3]:
# Instantiate one optimizer for the discriminator and another for the generator.
d_optimizer = keras.optimizers.Adam(learning_rate=0.00008)
g_optimizer = keras.optimizers.Adam(learning_rate=0.00004)

# Instantiate a loss function.
loss_fn = keras.losses.BinaryCrossentropy(from_logits=True)


@tf.function
def train_step(real_images,true_label,random_label0):
    # Sample random points in the latent space
    true_label=tf.cast(true_label,tf.float32)
    tf.random.set_seed(int(time.time()))
    np.random.seed(int(time.time()))

    random_latent_vectors = tf.random.normal(shape=(real_images.shape[0], latent_dim))
    # 随机类别
    random_label1=tf.random.shuffle(random_label0)
    random_latent_vectors=tf.concat([random_latent_vectors,random_label1],axis=-1)

    # Decode them to fake images
    generated_images = generator(random_latent_vectors)
    # Combine them with real images，最后为类别打乱组
    combined_images = tf.concat([generated_images, real_images], axis=0)
   
    # Assemble labels discriminating real from fake images
    # real:0 fake:1
    labels = tf.concat(
        [tf.ones((real_images.shape[0], 1)), tf.zeros((real_images.shape[0], 1))], axis=0
    )
    # Add random noise to the labels - important trick!
    labels += 0.05 * tf.random.uniform(labels.shape)
   
    # 类别信息输入
    class_layer=tf.concat([random_label1,true_label],axis=0)
    # class_layer=tf.reshape(class_layer,[class_layer.shape[0],1,1,10])
    # class_layer=tf.tile(class_layer,[1,28,28,1])
    # class_layer=tf.concat([combined_images,class_layer],axis=-1)
    for i in range(5):
    # Train the discriminator
        with tf.GradientTape() as tape:
            predictions = discriminator([combined_images,class_layer])
            d_loss = loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, discriminator.trainable_weights)
        d_optimizer.apply_gradients(zip(grads, discriminator.trainable_weights))
    
    # 打乱类别
    random_latent_vectors = tf.random.normal(shape=(real_images.shape[0], latent_dim))
    # 随机类别
    random_label1=tf.random.shuffle(random_label0)
    random_latent_vectors=tf.concat([random_latent_vectors,random_label1],axis=-1)
    # Decode them to fake images
    generated_images = generator(random_latent_vectors)
    # Combine them with real images，最后为类别打乱组
    combined_images = tf.concat([generated_images, real_images], axis=0)
    f_label=tf.random.shuffle(true_label)
    label_f=tf.abs(tf.cast(tf.equal(tf.argmax(true_label,axis=-1),tf.argmax(f_label,axis=-1)),dtype=tf.float32)-1)
    combind_label=tf.concat([rand_label1,f_label],axis=0)
    # class_layer=tf.tile(tf.reshape(combind_label,[combind_label.shape[0],1,1,10]),[1,28,28,1])
    # class_layer=tf.concat([combined_images,class_layer],axis=-1)
    for i in range(5):
        with tf.GradientTape() as tape:
            predictions = discriminator([combined_images,combind_label])
            d_loss = loss_fn(tf.concat([tf.ones(real_images.shape[0]),label_f],axis=0), predictions)
        grads = tape.gradient(d_loss, discriminator.trainable_weights)
        d_optimizer.apply_gradients(zip(grads, discriminator.trainable_weights))
    
    # Sample random points in the latent space
    random_latent_vectors = tf.random.normal(shape=(real_images.shape[0], latent_dim))
    random_label=tf.random.shuffle(random_label0)
    random_latent_vectors=tf.concat([random_latent_vectors,random_label],axis=-1)

    # Assemble labels that say "all real images"
    # real:0
    misleading_labels = tf.zeros((real_images.shape[0], 1))
    # class_layer=tf.tile(tf.reshape(random_label,[batch_size,1,1,10]),[1,28,28,1])
    # Train the generator (note that we should *not* update the weights
    # of the discriminator)!
    for i in range(20):
        with tf.GradientTape() as tape:
            predictions = discriminator([ generator(random_latent_vectors),random_label])
            g_loss = loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, generator.trainable_weights)
        g_optimizer.apply_gradients(zip(grads, generator.trainable_weights))
    return d_loss, g_loss, generated_images,random_label1

In [4]:
import os

# Prepare the dataset. We use both the training & test MNIST digits.
batch_size = 256
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
all_digits = np.concatenate([x_train, x_test])
all_label=keras.utils.to_categorical( np.concatenate([y_train,y_test]))
all_digits = all_digits.astype("float32") / 255.0
all_digits = np.reshape(all_digits, (-1, 28, 28, 1))
dataset = tf.data.Dataset.from_tensor_slices((all_digits,all_label))
dataset = dataset.shuffle(buffer_size=1024).batch(batch_size)


In [ ]:
@tf.function
def W_loss(real,fake,grade,lamda):
    loss=tf.reduce_mean(real,axis=0)-tf.reduce_mean(fake,axis=0)
    keras.regularizers.l2(1)(gra)


In [36]:

epochs = 200       # In practice you need at least 20 epochs to generate nice digits.
save_dir = "./"

for epoch in range(epochs):
    print("\nStart epoch", epoch)

    for step, [real_images,label] in enumerate(dataset):
        # break
        # Train the discriminator & generator on one batch of real images.
        rand_label1=keras.utils.to_categorical(np.random.randint(10,size=(batch_size,1)))
        d_loss, g_loss, generated_images,rand_label1= train_step(real_images,label,rand_label1)

        # Logging.
        if step % 20 == 0:
            # Print metrics
            print("discriminator loss at step %d: %.2f" % (step, d_loss))
            print("adversarial loss at step %d: %.2f" % (step, g_loss))

            # Save one generated image
            img = tf.keras.preprocessing.image.array_to_img(
                generated_images[0] * 255.0, scale=False
            )
            img.save(os.path.join(save_dir, "generated_img" + str(np.argmax(rand_label1,axis=-1)[0]) + ".png"))
            print(np.argmax(rand_label1,axis=-1)[0])
        # To limit execution time we stop after 10 steps.
        # Remove the lines below to actually train the model!
        # if step > 10:
        #     break

74
5
discriminator loss at step 20: 0.08
adversarial loss at step 20: 1.42
4
discriminator loss at step 40: 0.06
adversarial loss at step 40: 1.19
1
discriminator loss at step 60: 0.09
adversarial loss at step 60: 1.54
3
discriminator loss at step 80: 0.09
adversarial loss at step 80: 1.62
3
discriminator loss at step 100: 0.07
adversarial loss at step 100: 1.54
7
discriminator loss at step 120: 0.13
adversarial loss at step 120: 1.09
9
discriminator loss at step 140: 0.11
adversarial loss at step 140: 0.80
4
discriminator loss at step 160: 0.09
adversarial loss at step 160: 1.27
1
discriminator loss at step 180: 0.11
adversarial loss at step 180: 1.52
1
discriminator loss at step 200: 0.06
adversarial loss at step 200: 1.44
7
discriminator loss at step 220: 0.08
adversarial loss at step 220: 1.45
1
discriminator loss at step 240: 0.09
adversarial loss at step 240: 1.41
5
discriminator loss at step 260: 0.09
adversarial loss at step 260: 1.24
5

Start epoch 135
discriminator loss at st

KeyboardInterrupt: 

In [ ]:
for i in range(10):
    random_label1=keras.utils.to_categorical(np.random.randint(10,size=(batch_size,1)))
    print(np.argmax(random_label1,axis=-1)[0])

In [ ]:
a=tf.expand_dims(real_images[0,...],axis=0)
b=tf.reshape(label[4,...],[1,1,1,10])
b=tf.tile(b,[1,28,28,1])
c=tf.concat([a,b],axis=-1)
discriminator(c)

In [ ]:
plt.imshow(a[0])
print(label[4])

In [ ]:
if not os.path.exists("./generator_conditional") :
    os.mkdir("./generator_conditional")
generator.save("./generator_conditional")
if not os.path.exists("./discriminator_conditional") :
    os.mkdir("./discriminator_conditional")
discriminator.save("./discriminator_conditional")

In [ ]:
data1=np.random.random([1,32])
data2=np.eye(10)[[0]]
data=np.concatenate([data1,data2],axis=-1)
plt.imshow(generator(data)[0],cmap='gray')

In [2]:

a=tf.random.normal([2,3,4])
b=keras.regularizers.l2(0.1)(a)


In [13]:
gen=keras.models.load_model("./generator")
dis=keras.models.load_model("./discriminator")


[<tensorflow.python.keras.saving.saved_model.load.Dense object at 0x000001BFF7A8BC08>, <tensorflow.python.keras.saving.saved_model.load.LeakyReLU object at 0x000001BFF7A8C8C8>, <tensorflow.python.keras.saving.saved_model.load.Reshape object at 0x000001BFF7A8CEC8>, <tensorflow.python.keras.saving.saved_model.load.Conv2DTranspose object at 0x000001BFF7A8DBC8>, <tensorflow.python.keras.saving.saved_model.load.LeakyReLU object at 0x000001BFF7A8E388>, <tensorflow.python.keras.saving.saved_model.load.Conv2DTranspose object at 0x000001BFF7A8F048>, <tensorflow.python.keras.saving.saved_model.load.LeakyReLU object at 0x000001BFF7A8F808>, <tensorflow.python.keras.saving.saved_model.load.Conv2D object at 0x000001BFF7A90508>]


In [16]:
rand_value=tf.random.normal([1,128])
with tf.GradientTape() as tape:
    image=gen(rand_value)
    out=dis(image)
grade_1=tape.gradient(out,image)
grade_l2=tf.square(keras.regularizers.l2(1.0)(grade_1)-1.0)

